####  import related modules

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import random

In [2]:
#files_add=find_the_way(path,".csv")
#for i in files_add:
    #print(i)
    #os.remove(i)

#### File finder

In [3]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




#### Folder creator

In [4]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

path="captures_IoT-Sentinel"

#### Delete unnecessary files

In [5]:
#### Delete unnecessary files
def killthemall(path,key):
    them=find_the_way(path,key)
    for t in them:
        try:
            os.remove(t)
        except:
            print(f"error about delete {t} file")
#killthemall(path,".csv")

In [6]:
# find pcap file in path folder

pcaps=find_the_way(path,".pcap")
pcaps

['captures_IoT-Sentinel\\Aria\\Setup-A-1-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-2-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-3-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-4-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-5-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-1-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-10-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-11-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-12-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-13-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-14-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-15-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-2-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-3-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-4-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-5-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-6-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-7-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-8-STA.pcap',
 'capt

#### If your computer has difficulty processing large pcap files, you can split them using the following script

In [7]:
#for i in pcaps:
    #if os.path.getsize(i)//(1024*1024) >500:
        #temp=i.replace("\\","/")
        #command=f"editcap  -c 1000000 {temp} {temp.replace("pcaps/","pcaps/smaller_")}"
        #os.system(command)
        #print(f"{i} made smaller")
        #os.remove(temp)

In [8]:
### Renaming tools

In [9]:
# Feature list
feature=["eth.src",
    #'dstport',
  #'dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'ip.ttl','ipv6.hopopts.nxt',
  #'srcport',
  #'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.checksum.status',
  'udp.dstport',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"_ws.col.Protocol"]


# dictionary to convert string to category
strs={"http_request_method": {'DELETE': 1, 'GET': 2, 'M-SEARCH': 3, 'NOTIFY': 4, 'POST': 5, 'PUT': 6,'SUBSCRIBE':7, 'UNSUBSCRIBE':8,"HEAD":9},
"tcp_flags_str": {'·······AP··F': 1, '·······AP···': 2, '·······A·R··': 3, '·······A··S·': 4,
    '·······A···F': 5, '·······A····': 6, '·········R··': 7,  '··········S·': 8,'····CE····S·':9, "····C··AP···":10,"···········F":11}}

# string to category feature list
str_cols=["http.request.method",
"tcp.flags.str"]

# MAC address of devices and their names. It will use for labelling
macs=MAC_list={# IoT devices captures MAC and Device names
'00:17:88:24:76:ff': 'Hue-Device',
 '00:1a:22:03:cb:be': 'MAXGateway',
 '00:1a:22:05:c4:2e': 'HomeMaticPlug',
 '00:24:e4:24:80:2a': 'Withings',
 '00:b5:6d:06:08:ba': 'unknown',
 '1c:5f:2b:aa:fd:4e': 'D-LinkDevice',
 '20:f8:5e:ca:91:52': 'Aria',
 '24:77:03:7c:ea:dc': 'unknown',
 '28:b2:bd:c3:41:79': 'unknown',
 '38:0b:40:ef:85:41': 'unknown',
 '50:c7:bf:00:c7:03': 'TP-LinkPlugHS110',
 '50:c7:bf:00:fc:a3': 'TP-LinkPlugHS100',
 '3c:49:37:03:17:db': 'EdnetCam',
 '3c:49:37:03:17:f0': 'EdnetCam',
 '5c:cf:7f:06:d9:02': 'iKettle2',
 '5c:cf:7f:07:ae:fb': 'SmarterCoffee',
 '6c:72:20:c5:17:5a': 'D-LinkWaterSensor',
 '74:da:38:23:22:7b': 'EdimaxPlug2101W',
 '74:da:38:4a:76:49': 'EdimaxPlug1101W',
 '74:da:38:80:79:fc': 'EdimaxCam',
 '74:da:38:80:7a:08': 'EdimaxCam',
 '84:18:26:7b:5f:6b': 'Lightify',
 '90:8d:78:a8:e1:43': 'D-LinkSensor',
 '90:8d:78:a9:3d:6f': 'D-LinkSwitch',
 '90:8d:78:dd:0d:60': 'D-LinkSiren',
 '94:10:3e:34:0c:b5': 'WeMoSwitch',
 '94:10:3e:35:01:c1': 'WeMoSwitch',
 '94:10:3e:41:c2:05': 'WeMoInsightSwitch',
 '94:10:3e:42:80:69': 'WeMoInsightSwitch',
 '94:10:3e:cd:37:65': 'WeMoLink',
 'ac:cf:23:62:3c:6e': 'EdnetGateway',
 'b0:c5:54:1c:71:85': 'D-LinkDayCam',
 'b0:c5:54:25:5b:0e': 'D-LinkCam',
 'bc:f5:ac:f4:c0:9d': 'unknown'}

In [10]:
dubs=['ip.flags.df', 'ip.len', 'ip.proto', 'ip.ttl', 'tls.record.length']
hexvalues=['tcp.flags']

In [11]:
# port number to port classes
def port_class(port):
    port_list=[0,53,67,68,80,123,443,1900,5353,49153]# private port list (0-Reserved,53-DNS, 67-BOOTP server, 68-BOOTP client...)
    if port in port_list: #Is the port number in the list?
        return port_list.index(port)+1 # return the port's index number in the list (actually with index+1)
    elif 0 <= port <= 1023: # return 11 if the port number is in the range 0 :1023
        return 11
    elif  1024 <= port <= 49151 : # return 12 if the port number is in the range 1024:49151
        return 12
    elif 49152 <=port <= 65535 :# return 13 if the port number is in the range 49152:65535
        return 13
    else:# return 0 if no previous conditions are met
        return 0
    

In [12]:
# hexadecimal to decimal function
def comma_cleaner(val):
    clean=[]
    for v in val:
        if "," in str(v):
            temp=v.split(",")
            temp=temp[0]
        else:
            temp=v
        clean.append(temp)  
    return clean


In [13]:
# features with hexadecimal values
# function that holds the first value and discards the second value for repeated values
def hex2dec(val):
    clean=[]
    for v in val:
        try:
            clean.append(int(v, 16))
        except:
            clean.append(v)  
    return clean


In [14]:
path="captures_IoT-Sentinel"
pcaps=find_the_way(path,".pcap")
pcaps

['captures_IoT-Sentinel\\Aria\\Setup-A-1-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-2-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-3-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-4-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-A-5-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-1-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-10-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-11-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-12-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-13-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-14-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-15-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-2-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-3-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-4-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-5-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-6-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-7-STA.pcap',
 'captures_IoT-Sentinel\\Aria\\Setup-C-8-STA.pcap',
 'capt

In [15]:
#### pcap to csv using wireshark
#### Feature extraction main part

for  pp in tqdm(pcaps):
    name=pp.replace("\\","/")
   
    line=f"tshark -r {name} -E header=y -E separator=\"|\" -T fields -e "
    for i in feature:
        line=line+ i + " -e "
    output=name.replace(".pcap",".csv")
    line= line[:-3] + f"> {output}"
    os.system(line)

100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [02:28<00:00,  3.70it/s]


In [16]:
#### General corrections in CSV files (payload entropy, port class, merging of IPv4 and IPv6 properties, Labelling, etc.)
#### nan values filled with -9999
#### inf values filled with max(uint32)
files_add=find_the_way(path,".csv")
for name in tqdm(files_add):
    df=pd.read_csv(name,sep="|")
    for d in dubs:
        temp=df[d].values
        temp=comma_cleaner(temp)
        df[d]=temp
    for h in hexvalues:
        temp=df[h].values
        temp=hex2dec(temp)
        df[h]=temp

    for i in str_cols:
        temp=i.replace(".","_")
        val=df[i]
        val=val.replace(strs[temp])
        temp=hex2dec(temp)
        df[i]=val

    temp=[]


    df['dstport'] = df['udp.dstport'].combine_first(df['udp.dstport'])
    df['srcport'] = df['udp.srcport'].combine_first(df['udp.srcport'])
    
    temp=[]
    for i in (df['dstport'].values):
        temp.append(port_class(i))
    df['dstport_class']=temp
    
    temp=[]
    for i in (df['srcport'].values):
        temp.append(port_class(i))
    df['srcport_class']=temp


    df['dstport'] = df['udp.dstport'].combine_first(df['udp.dstport'])
    df['srcport'] = df['udp.srcport'].combine_first(df['udp.srcport'])
    #df['ip.dst'] = df['ip.dst'].combine_first(df['ipv6.dst'])
   # df['ip.src'] = df['ip.src'].combine_first(df['ipv6.src'])
    
    df['ip.len'] = df['ip.len'].combine_first(df['ipv6.hopopts.nxt'])


    df=df.fillna(-9999)
    df["Label"]=df["eth.src"].replace(macs)
    #del df["eth.src"]
    df.to_csv(name,index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [00:16<00:00, 33.16it/s]


In [17]:
train=['captures_IoT-Sentinel/Aria/Setup-A-1-STA.csv',
       'captures_IoT-Sentinel/WeMoLink/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-11-STA.csv',
       'captures_IoT-Sentinel/Lightify/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-12-STA.csv',
       'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-4-STA.csv',
       'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-4-STA.csv',
       'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-A-1-STA.csv',
       'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-19-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-B-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-B-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-B-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-B-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-B-5-STA.csv',
       'captures_IoT-Sentinel/D-LinkSiren/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-19-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-16-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-17-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-14-STA.csv',
       'captures_IoT-Sentinel/EdnetGateway/Setup-C-17-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-3-STA.csv',
       'captures_IoT-Sentinel/EdnetCam1/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/EdnetCam1/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-16-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-17-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-16-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-17-STA.csv', 
       'captures_IoT-Sentinel/iKettle2/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-11-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-12-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-13-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-14-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-15-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-2-STA.csv',
       'captures_IoT-Sentinel/HueBridge/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-8-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-12-STA.csv',
       'captures_IoT-Sentinel/SmarterCoffee/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-11-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-12-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-13-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-14-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-15-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-13-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-14-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-15-STA.csv',
       'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-A-3-STA.csv', 
       'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-8-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-1-STA.csv', 
       'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-11-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-8-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-13-STA.csv',
       'captures_IoT-Sentinel/Withings/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-16-STA.csv', 
       'captures_IoT-Sentinel/MAXGateway/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-8-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-17-STA.csv', 
       'captures_IoT-Sentinel/WeMoSwitch/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-10-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-8-STA.csv', 
       'captures_IoT-Sentinel/D-LinkCam/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-B-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-B-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-B-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-B-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-B-5-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-2-STA.csv']

validation=[ 'captures_IoT-Sentinel/Aria/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-2-STA.csv', 
            'captures_IoT-Sentinel/EdnetCam1/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-3-STA.csv',
            'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-16-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-17-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam1/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-A-2-STA.csv', 
            'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-4-STA.csv', 
            'captures_IoT-Sentinel/Withings/Setup-C-18-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-19-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-4-STA.csv', 
            'captures_IoT-Sentinel/WeMoSwitch/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch2/Setup-C-1-STA.csv', 
            'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-2-STA.csv',
            'captures_IoT-Sentinel/iKettle2/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-8-STA.csv',
            'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-19-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-20-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-3-STA.csv',
            'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-C-12-STA.csv', 
            'captures_IoT-Sentinel/D-LinkSiren/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-16-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-17-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-4-STA.csv', 
            'captures_IoT-Sentinel/Lightify/Setup-C-19-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-20-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-3-STA.csv',
            'captures_IoT-Sentinel/HueBridge/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-4-STA.csv',
            'captures_IoT-Sentinel/EdnetGateway/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-13-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-14-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-15-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-16-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-19-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-20-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch2/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch2/Setup-C-3-STA.csv',
            'captures_IoT-Sentinel/D-LinkCam/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-2-STA.csv',
            'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-2-STA.csv', 
            'captures_IoT-Sentinel/SmarterCoffee/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-8-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-A-9-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-4-STA.csv',
            'captures_IoT-Sentinel/WeMoLink/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-6-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-7-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-8-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-A-9-STA.csv', 
            'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-9-STA.csv',
            'captures_IoT-Sentinel/MAXGateway/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch2/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch2/Setup-C-2-STA.csv']


test=[ 'captures_IoT-Sentinel/Aria/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/Aria/Setup-C-9-STA.csv',
      'captures_IoT-Sentinel/Withings/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/Withings/Setup-C-9-STA.csv', 
      'captures_IoT-Sentinel/WeMoSwitch/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch/Setup-C-6-STA.csv', 
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkCam/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch2/Setup-C-4-STA.csv', 
      'captures_IoT-Sentinel/EdnetCam2/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-7-STA.csv', 
      'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug1101W/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/EdimaxPlug2101W/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/EdnetCam2/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-C-3-STA.csv',
      'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch2/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch2/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/WeMoInsightSwitch2/Setup-C-5-STA.csv', 
 'captures_IoT-Sentinel/iKettle2/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-C-4-STA.csv',
      'captures_IoT-Sentinel/D-LinkDayCam/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkDayCam/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-6-STA.csv', 
      'captures_IoT-Sentinel/D-LinkHomeHub/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/D-LinkHomeHub/Setup-A-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-5-STA.csv', 
      'captures_IoT-Sentinel/D-LinkSiren/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/D-LinkSiren/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkSwitch/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkSensor/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/D-LinkDoorSensor/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/iKettle2/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS110/Setup-C-9-STA.csv', 
      'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/TP-LinkPlugHS100/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-7-STA.csv', 
      'captures_IoT-Sentinel/EdnetGateway/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/EdnetGateway/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/HomeMaticPlug/Setup-C-9-STA.csv',
      'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-10-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-11-STA.csv',
 'captures_IoT-Sentinel/D-LinkWaterSensor/Setup-C-12-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-C-1-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/EdimaxCam2/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/MAXGateway/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-C-2-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-C-3-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/SmarterCoffee/Setup-C-5-STA.csv', 
      'captures_IoT-Sentinel/Lightify/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/Lightify/Setup-C-9-STA.csv', 
      'captures_IoT-Sentinel/HueBridge/Setup-A-1-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-A-2-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-A-3-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-A-4-STA.csv',
 'captures_IoT-Sentinel/HueBridge/Setup-A-5-STA.csv',
'captures_IoT-Sentinel/HueSwitch/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-6-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-7-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-8-STA.csv',
 'captures_IoT-Sentinel/HueSwitch/Setup-C-9-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-C-4-STA.csv',
 'captures_IoT-Sentinel/WeMoLink/Setup-C-5-STA.csv',
 'captures_IoT-Sentinel/WeMoSwitch2/Setup-C-5-STA.csv']

In [27]:
df_list={"Train_GeMoDI.csv":train,"Validation_GeMoDI.csv":validation,"Test_GeMoDI.csv":test}

for name in df_list:

    df=pd.read_csv(df_list[name][0])
    col_names=list(df.columns)
    empty = pd.DataFrame(columns=col_names)
    empty.to_csv(name, mode="w", index=False)#,header=False)
    for iii in tqdm(df_list[name]):
        df=pd.read_csv(iii)
        df.to_csv(name, mode="a", index=False,header=False)
        df=pd.read_csv(name)

100%|████████████████████████████████████████████████████████████████████████████████| 137/137 [00:17<00:00,  7.75it/s]


In [28]:
df

,eth.src,http.chat,http.notification,http.request.method,ip.flags.df,ip.len,ip.proto,ip.ttl,ipv6.hopopts.nxt,tcp.ack,...,udp.dstport,udp.srcport,udp.time_delta,udp.time_relative,_ws.col.Protocol,dstport,srcport,dstport_class,srcport_class,Label
0,20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.000000,-9999.000000,EAPOL,-9999.0,-9999.0,0,0,Aria
1,20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.000000,-9999.000000,EAPOL,-9999.0,-9999.0,0,0,Aria
2,20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,0.0,328.0,17.0,64.0,-9999.0,-9999.0,...,67.0,68.0,0.000000,0.000000,DHCP,67.0,68.0,3,4,Aria
3,00:b5:6d:06:08:ba,-9999.0,-9999.0,-9999.0,0.0,328.0,17.0,64.0,-9999.0,-9999.0,...,68.0,67.0,0.000000,0.000000,DHCP,68.0,67.0,4,3,unknown
4,20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,0.0,328.0,17.0,64.0,-9999.0,-9999.0,...,67.0,68.0,0.006404,0.006404,DHCP,67.0,68.0,3,4,Aria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47591,94:10:3e:34:0c:b5,1.0,1.0,4.0,1.0,457.0,17.0,4.0,-9999.0,-9999.0,...,1900.0,3085.0,0.000157,0.134765,SSDP,1900.0,3085.0,8,12,WeMoSwitch
47592,94:10:3e:34:0c:b5,1.0,1.0,4.0,1.0,463.0,17.0,4.0,-9999.0,-9999.0,...,1900.0,3085.0,0.000162,0.134927,SSDP,1900.0,3085.0,8,12,WeMoSwitch
47593,94:10:3e:34:0c:b5,1.0,1.0,4.0,1.0,471.0,17.0,4.0,-9999.0,-9999.0,...,1900.0,3085.0,0.000162,0.135089,SSDP,1900.0,3085.0,8,12,WeMoSwitch
47594,00:b5:6d:06:08:ba,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.000000,-9999.000000,ARP,-9999.0,-9999.0,0,0,unknown


In [29]:
for i in df_list:
    print(i)
    df=pd.read_csv(i)
    df=df[df["Label"]!="unknown"]
    df.to_csv(i,index=False)
    print(df.groupby("Label").size())

Train_GeMoDI.csv
Label
Aria                   259
D-LinkCam             3849
D-LinkDayCam           621
D-LinkDevice          6082
D-LinkSensor          3461
D-LinkSiren           3197
D-LinkSwitch          3314
D-LinkWaterSensor     3232
EdimaxCam              438
EdimaxPlug1101W        711
EdimaxPlug2101W        489
EdnetCam               187
EdnetGateway           398
HomeMaticPlug          181
Hue-Device           16391
Lightify              2085
MAXGateway             329
SmarterCoffee           96
TP-LinkPlugHS100       359
TP-LinkPlugHS110       347
WeMoInsightSwitch     2807
WeMoLink              3271
WeMoSwitch            2466
Withings               376
iKettle2                94
dtype: int64
Validation_GeMoDI.csv
Label
Aria                  131
D-LinkCam            1273
D-LinkDayCam          309
D-LinkDevice         2230
D-LinkSensor         1599
D-LinkSiren          1574
D-LinkSwitch         1670
D-LinkWaterSensor    1688
EdimaxCam             223
EdimaxPlug1101W       261
E

In [22]:
df

,"eth.src,http.chat,http.notification,http.request.method,ip.flags.df,ip.len,ip.proto,ip.ttl,ipv6.hopopts.nxt,tcp.ack,tcp.analysis.ack_rtt,tcp.analysis.bytes_in_flight,tcp.analysis.initial_rtt,tcp.analysis.push_bytes_sent,tcp.completeness,tcp.dstport,tcp.flags,tcp.flags.push,tcp.flags.str,tcp.flags.syn,tcp.hdr_len,tcp.len,tcp.nxtseq,tcp.srcport,tcp.stream,tcp.time_delta,tcp.time_relative,tcp.window_size,tcp.window_size_scalefactor,tcp.window_size_value,tls.record.length,udp.checksum.status,udp.dstport,udp.srcport,udp.time_delta,udp.time_relative,_ws.col.Protocol,dstport,srcport,dstport_class,srcport_class,Label"
0,"20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,-999..."
1,"20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,-999..."
2,"20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,0.0,..."
3,"00:b5:6d:06:08:ba,-9999.0,-9999.0,-9999.0,0.0,..."
4,"20:f8:5e:ca:91:52,-9999.0,-9999.0,-9999.0,0.0,..."
...,...
98661,"00:b5:6d:06:08:ba,-9999.0,-9999.0,-9999.0,0.0,..."
98662,"94:10:3e:35:01:c1,-9999.0,-9999.0,-9999.0,1.0,..."
98663,"00:b5:6d:06:08:ba,-9999.0,-9999.0,-9999.0,0.0,..."
98664,"00:b5:6d:06:08:ba,-9999.0,-9999.0,-9999.0,-999..."


  0%|                                                                                 | 1/1100 [00:00<00:25, 42.31it/s]


KeyError: 'ip.flags.df'

In [44]:
files_add=find_the_way(path,".csv")


['iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_17_23_20.68s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_18_38_57.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_19_39_01.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_32_13.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_51_27.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_13_35.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_35_48.64s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_57_55.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_20_02.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_42_11.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_17_04_20.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28

# Move CSV output to related folders

In [46]:
folder("csvs")

In [47]:
folders=[]
for p in pcaps:
    newname=p.replace("\\","/")
    newname=newname.split("/")
    if newname[1] not in folders:
        folders.append(newname[1])
folders

['uk', 'uk-vpn', 'us', 'us-vpn']

In [48]:
for f in folders:
    print(f)
    address=f"{path}/{f}/"
    pcaps=find_the_way(address,".csv")
    folder(address.replace("iot-data","csvs"))
    for p in tqdm(pcaps):
        newname=p.replace("\\","/")
        #newname=newname.replace("./iot-data","./pcaps")
        newname=newname[len(address):].replace("/","@")
        newname=f"{address}{newname}"
        newname=newname.replace("iot-data","csvs")
        #print(p,"\n",newname)
        os.rename(p,newname)


uk


100%|█████████████████████████████████████████████████████████████████████████████| 8023/8023 [00:09<00:00, 807.95it/s]


uk-vpn


100%|████████████████████████████████████████████████████████████████████████████| 9175/9175 [00:09<00:00, 1011.84it/s]


us


100%|██████████████████████████████████████████████████████████████████████████| 10324/10324 [00:09<00:00, 1125.62it/s]


us-vpn


100%|██████████████████████████████████████████████████████████████████████████| 10222/10222 [00:08<00:00, 1140.03it/s]


# Merge Small CSVs

### if you have split the pcap files into small sizes, you can use this section to merge their csv output

In [49]:
folders= ['uk', 'uk-vpn', 'us', 'us-vpn']
devices=[ 'appletv',
 'blink-camera',
 'blink-security-hub',
 'echodot',
 'echospot',
 'echoplus',
 'firetv',
 'google-home-mini',
 'insteon-hub',
 'lightify-hub',
 'magichome-strip',
 'nest-tstat',
 'ring-doorbell',
 'roku-tv',
 'samsungtv-wired',
 'sengled-hub',
 'smartthings-hub',
 'sousvide',
 't-philips-hub',
 'tplink-bulb',
 'tplink-plug',
 't-wemo-plug',
 'wansview-cam-wired',
 'xiaomi-hub',
 'yi-camera']

In [50]:
for f in folders:
    print(f)
    ths = open(f"./csvs/{f}.csv", "w")
    flag=1
    them=find_the_way(f"./csvs/{f}/",".csv")
    for t in tqdm(them):
        #print(t)
        with open(t, "r") as file:
            while True:
                line=file.readline()
                if line=="":break
                k=line[:-1]
                k=k.split(",")
                k=k[-1]
                if not line.startswith("eth"):
                    #print(line)
                    if "ARP" not in line:
                        if k in devices:
                            ths.write(line)
                        #print(line)
                        #break
                else:
                    if flag:
                        ths.write(line)
                        flag=0
        os.remove(t)
    ths.close()

uk


100%|█████████████████████████████████████████████████████████████████████████████| 8023/8023 [00:41<00:00, 192.81it/s]


uk-vpn


100%|█████████████████████████████████████████████████████████████████████████████| 9175/9175 [00:44<00:00, 207.63it/s]


us


100%|███████████████████████████████████████████████████████████████████████████| 10324/10324 [00:53<00:00, 191.31it/s]


us-vpn


100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:47<00:00, 215.68it/s]


In [52]:
for f in folders:

    print(f)
    df=pd.read_csv(f"./csvs/{f}.csv",usecols=["Label"])
    temp=df.groupby("Label").size()
    print(len(temp),temp)
    print("#"*120,"\n\n\n")

uk
25 Label
appletv                17977
blink-camera           83855
blink-security-hub      3352
echodot                31471
echoplus               27440
echospot               65150
firetv                 10487
google-home-mini       19209
insteon-hub               33
lightify-hub            3274
magichome-strip         1138
nest-tstat             25651
ring-doorbell         820444
roku-tv                 5876
samsungtv-wired        27148
sengled-hub             3370
smartthings-hub        21271
sousvide               15842
t-philips-hub          71515
t-wemo-plug            32546
tplink-bulb            14580
tplink-plug             5965
wansview-cam-wired    802384
xiaomi-hub              6394
yi-camera             261276
dtype: int64
######################################################################################################################## 



uk-vpn
25 Label
appletv                13775
blink-camera           92434
blink-security-hub      3863
echodot               

# Alternative method to shrink CSV
# CREATE SMALLER CSVs (1/10)

#### Using pandas on large CSV files can sometimes cause problems. In the following example I have developed a system that can sample very large files quickly and without problems.

In [4]:
folder("small")
folders= ['uk', 'uk-vpn', 'us', 'us-vpn']

In [6]:
for f in folders:
    print(f)
    df=pd.read_csv(f"./csvs/{f}.csv")#,usecols=["Label"])
    df = df.sample(n = len(df)//10) 
    df.to_csv(f"./small/{f}.csv",index=False)
    print(f,df.groupby("Label").size())
    print("__________________________________________________________________")

uk
uk Label
appletv                1797
blink-camera           8469
blink-security-hub      332
echodot                3065
echoplus               2750
echospot               6390
firetv                 1046
google-home-mini       1906
insteon-hub               3
lightify-hub            318
magichome-strip         117
nest-tstat             2550
ring-doorbell         81823
roku-tv                 604
samsungtv-wired        2691
sengled-hub             383
smartthings-hub        2159
sousvide               1560
t-philips-hub          7269
t-wemo-plug            3293
tplink-bulb            1478
tplink-plug             600
wansview-cam-wired    80410
xiaomi-hub              610
yi-camera             26141
dtype: int64
__________________________________________________________________
uk-vpn
uk-vpn Label
appletv                1392
blink-camera           9385
blink-security-hub      405
echodot                2272
echoplus               3587
echospot               8465
firetv              

### without sampling devices with a sample count below 100

In [8]:
def reduce_dataset(df, reduction_ratio):
    reduced_df = pd.DataFrame()
    for label, count in label_counts.items():
        if count >= 100:
            reduced_count = int(count * reduction_ratio)
        else:
            reduced_count = count
        label_df = df[df['Label'] == label].sample(n=reduced_count, random_state=42)
        reduced_df = pd.concat([reduced_df, label_df])
    return reduced_df

for f in folders:
    print(f)
    
    df=pd.read_csv(f"./csvs/{f}.csv")#,usecols=["Label"])
    print(f,df.groupby("Label").size())
    label_counts = df['Label'].value_counts().to_dict()
    reduction_ratio = 0.1  # Küçültme oranı
    reduced_df = reduce_dataset(df, reduction_ratio)
    reduced_df.to_csv(f"./small/{f}.csv",index=False)
    print(f,reduced_df.groupby("Label").size())
    print("__________________________________________________________________")

uk
uk Label
appletv                17977
blink-camera           83855
blink-security-hub      3352
echodot                31471
echoplus               27440
echospot               65150
firetv                 10487
google-home-mini       19209
insteon-hub               33
lightify-hub            3274
magichome-strip         1138
nest-tstat             25651
ring-doorbell         820444
roku-tv                 5876
samsungtv-wired        27148
sengled-hub             3370
smartthings-hub        21271
sousvide               15842
t-philips-hub          71515
t-wemo-plug            32546
tplink-bulb            14580
tplink-plug             5965
wansview-cam-wired    802384
xiaomi-hub              6394
yi-camera             261276
dtype: int64
uk Label
appletv                1797
blink-camera           8385
blink-security-hub      335
echodot                3147
echoplus               2744
echospot               6515
firetv                 1048
google-home-mini       1920
insteon-hub      